In [1]:
import pandas  as  pd

## 1. 파일불러오기

In [2]:
df = pd.read_csv("raw_data/20.5개시도_공원현황.csv")
df.isnull().sum()

park_nm     0
park_gbn    0
lon         0
lat         0
dtype: int64

In [8]:
# 추후 반복문에서 replace로 바꾸기 위해
df["관할서"] = "모름"


## 2. json파일, 위도경도 이용하여 관할서 지정

In [3]:
# 위도경도 데이터 zip
location = list(zip(df["lon"], df["lat"]))

In [4]:
# geopandas, Point  라이브러리
import geopandas as gpd
from shapely.geometry import Point, Polygon

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [5]:
# 위도경도 데이터 Point로 만들어주기
st_list = []
for i in location:
    test_code = Point(i)
    st_list.append(test_code)

In [11]:
#관할서 경계데이터 geopandas로 읽어오기
df_js = gpd.read_file('raw_data/1.5개시도_경찰서_관할경계.geojson', encoding='cp949')
df_js.head()

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."
3,창원중부경찰서,경남청,"MULTIPOLYGON (((128.60966 35.15093, 128.60956 ..."
4,마산동부경찰서,경남청,"MULTIPOLYGON (((128.62696 35.21714, 128.62695 ..."


In [13]:
# 빠른 반복문 진행을 위해 변수지정
df_js_list = df_js["geometry"]
df_js_name = df_js["NAME"]

In [7]:
#반복문진행률
from tqdm import tqdm

In [26]:
# 위도경도 데이터가 관할경계데이터에 포함되면  관할서의 이름값을 df관할서의 "모름"을 관할서 이름으로 바꿔라  
for z in tqdm(range(41)):
    for i in range(len(df)):
        if st_list[i].within(df_js_list[z]):
            df.iloc[i,-1] = df.iloc[i,-1].replace("모름",df_js_name[z])
            
        else:
            pass

100%|██████████| 41/41 [01:48<00:00,  2.65s/it]


In [28]:
#파일저장
df.to_csv("데이터별_관할서리스트/공원현황_관할서리스트")

In [30]:
pd.read_csv("데이터별_관할서리스트/공원현황_관할서리스트", index_col = 0)

,park_nm,park_gbn,lon,lat,관할서
0,도램샘 어린이공원(어1-13),어린이공원,127.253744,36.520719,세종경찰서
1,꾸러기 어린이공원(어1-14),어린이공원,127.241635,36.502642,세종경찰서
2,푸른꿈 어린이공원(어1-15),어린이공원,127.262852,36.519766,세종경찰서
3,방울새 어린이공원(어1-18),어린이공원,127.268955,36.496070,세종경찰서
4,푸른하늘 어린이공원(어1-19),어린이공원,127.245986,36.510491,세종경찰서
...,...,...,...,...,...
2824,쌍문근린공원,근린공원,127.029088,37.654192,서울도봉경찰서
2825,월천근린공원,근린공원,127.052319,37.647271,서울도봉경찰서
2826,누 원,어린이공원,127.050046,37.680396,서울도봉경찰서
2827,새동네,어린이공원,127.043172,37.688975,서울도봉경찰서
